In this notebook, we take a closer look into how the embedding of LCEGP changes as we
add more samples and re-train the model.

In the experiments, we observe some wild swings in the LCEGP performance between each
model fit. We want to understand why that happens, and figure out if we can get rid of
the problem.

In [1]:
from typing import Tuple
import torch
from botorch import fit_gpytorch_model
from botorch.models import SingleTaskGP
from botorch.models.transforms import Standardize
from botorch.test_functions import Griewank
from botorch.utils.transforms import unnormalize
from gpytorch import ExactMarginalLogLikelihood
from torch import Tensor

from contextual_rs.models.lce_gp import LCEGP
from contextual_rs.models.custom_fit import custom_fit_gpytorch_model
from contextual_rs.models.contextual_independent_model import ContextualIndependentModel


function = Griewank(dim=2, negate=True, noise_std=5.0)
function.bounds[0, :].fill_(-5)
function.bounds[0, :].fill_(5)


def eval_function(X: Tensor, evaluate_true: bool = False) -> Tensor:
    if evaluate_true:
        return function.evaluate_true(unnormalize(X, function.bounds))
    else:
        return function(unnormalize(X, function.bounds))


In [2]:
num_arms = 4
num_contexts = 6
num_alternatives = num_arms * num_contexts
num_full_train = 5
arm_set = torch.arange(0, num_arms).view(-1, 1)
context_set = torch.linspace(0, 1, num_contexts).view(-1, 1)

lcegp_train_X = torch.cat(
    [
        arm_set.view(-1, 1, 1).repeat(1, num_contexts, 1),
        context_set.repeat(num_arms, 1, 1),
    ], dim=-1
).view(-1, 2).repeat(num_full_train, 1)
st_train_X = lcegp_train_X.clone()
st_train_X[..., 0] = st_train_X[..., 0] / (num_arms - 1)
indep_train_X = lcegp_train_X.clone()
indep_train_X[..., 1] = indep_train_X[..., 1] * (num_contexts - 1)
train_Y = eval_function(st_train_X).view(-1, 1)

true_mean = eval_function(
    st_train_X[:num_alternatives], evaluate_true=True
).view(num_arms, num_contexts)

lcegp = LCEGP(
    lcegp_train_X, train_Y, [0], outcome_transform=Standardize(m=1)
)
mll = ExactMarginalLogLikelihood(lcegp.likelihood, lcegp)
custom_fit_gpytorch_model(mll)

stgp = SingleTaskGP(
    st_train_X, train_Y, outcome_transform=Standardize(m=1)
)
mll = ExactMarginalLogLikelihood(stgp.likelihood, stgp)
fit_gpytorch_model(mll)

indep_model = ContextualIndependentModel(
    indep_train_X, train_Y.squeeze(-1)
)

Let's take a look at the posterior means as predicted by these models.

In [3]:
lcegp_pm = lcegp.posterior(lcegp_train_X[:num_alternatives]).mean.view(num_arms, num_contexts)
st_pm = stgp.posterior(st_train_X[:num_alternatives]).mean.view(num_arms, num_contexts)
indep_pm = indep_model.means.view(num_arms, num_contexts)

for arm_idx in range(num_arms):
    print(f"Arm idx: {arm_idx}")
    print(f"True mean: {true_mean[arm_idx]}")
    print(f"Indep mean: {indep_pm[arm_idx]}")
    print(f"LCEGP mean: {lcegp_pm[arm_idx]}")
    print(f"ST mean: {st_pm[arm_idx]}")


Arm idx: 0
True mean: tensor([ 1.0125,  4.8503, 15.7685, 33.7673, 58.8465, 91.0062])
Indep mean: tensor([ -1.9953,  -7.2004, -17.4813, -30.2895, -60.3034, -91.6836])
LCEGP mean: tensor([ -2.1314,  -6.4410, -16.5430, -32.9897, -59.8297, -91.6104],
       grad_fn=<SelectBackward>)
ST mean: tensor([ -2.1464,  -7.2155, -16.3478, -32.2198, -59.4672, -91.6482],
       grad_fn=<SelectBackward>)
Arm idx: 1
True mean: tensor([ 11.3424,  15.1801,  26.0984,  44.0971,  69.1764, 101.3361])
Indep mean: tensor([  -9.0395,  -16.4518,  -24.2740,  -45.4381,  -69.1548, -101.6101])
LCEGP mean: tensor([ -10.8063,  -15.0129,  -25.7578,  -43.0279,  -70.0538, -101.2419],
       grad_fn=<SelectBackward>)
ST mean: tensor([  -9.9959,  -14.9000,  -25.7136,  -44.2292,  -70.0255, -100.9696],
       grad_fn=<SelectBackward>)
Arm idx: 2
True mean: tensor([ 41.3403,  45.1780,  56.0963,  74.0950,  99.1743, 131.3340])
Indep mean: tensor([ -45.0957,  -44.0481,  -58.8242,  -75.4303, -101.1235, -128.0902])
LCEGP mean: tens

LCEGP seems to deviate from the empirical mean slightly more than others. Let's fit
each model on the same data multiple times and see how much the posterior mean varies
between attempts.

We will not re-fit the independent model since it is deterministic (sample mean).

In [4]:
def fit_replications(replications: int, fit_tries: int = 1) -> Tuple[Tensor, Tensor]:
    lcegp_pms = torch.zeros(replications, num_arms, num_contexts)
    st_pms = torch.zeros(replications, num_arms, num_contexts)
    for rep in range(replications):
        lcegp = LCEGP(
            lcegp_train_X, train_Y, [0], outcome_transform=Standardize(m=1)
        )
        mll = ExactMarginalLogLikelihood(lcegp.likelihood, lcegp)
        custom_fit_gpytorch_model(mll, num_retries=fit_tries)

        stgp = SingleTaskGP(
            st_train_X, train_Y, outcome_transform=Standardize(m=1)
        )
        mll = ExactMarginalLogLikelihood(stgp.likelihood, stgp)
        fit_gpytorch_model(mll)

        lcegp_pms[rep] = lcegp.posterior(
            lcegp_train_X[:num_alternatives]
        ).mean.view(num_arms, num_contexts)
        st_pms[rep] = stgp.posterior(
            st_train_X[:num_alternatives]
        ).mean.view(num_arms, num_contexts)
    return lcegp_pms, st_pms

In [5]:
lcegp_pms, st_pms = fit_replications(30)
lcegp_stds = lcegp_pms.std(dim=0)
st_stds = st_pms.std(dim=0)

lcegp_diffs = lcegp_pms - indep_pm
st_diffs = st_pms - indep_pm

lcegp_mse = lcegp_diffs.pow(2).mean(dim=0)
st_mse = st_diffs.pow(2).mean(dim=0)

print(f"lcegp avg std: {lcegp_stds.mean()}")
print(f"st avg std: {st_stds.mean()}")

print(f"lcegp avg mse: {lcegp_mse.mean()}")
print(f"st avg mse: {st_mse.mean()}")

lcegp_mse_normalized = lcegp_mse.sqrt() / indep_pm.abs()
st_mse_normalized = st_mse.sqrt() / indep_pm.abs()

print(f"lcegp avg normalized mse: {lcegp_mse_normalized.mean()}")
print(f"st avg normalized mse: {st_mse_normalized.mean()}")

lcegp avg std: 0.5305986404418945
st avg std: 0.0
lcegp avg mse: 2.3495187759399414
st avg mse: 1.4980558156967163
lcegp avg normalized mse: 0.04930578172206879
st avg normalized mse: 0.029212482273578644


Depending on the data, LCEGP may have significantly larger MSE (re-run notebook if
not obvious). It also clearly shows some deviation in posterior mean between repetitions,
which STGP does not have. This suggests that we get "local" fit for the
hyper-parameters, which depends on the initialization, rather than a global one.

Before looking into the latent covariance directly, let's check how re-tries affect the
 behavior.

In [6]:
lcegp_pms, st_pms = fit_replications(30, fit_tries=10)
lcegp_stds = lcegp_pms.std(dim=0)
st_stds = st_pms.std(dim=0)

lcegp_diffs = lcegp_pms - indep_pm
st_diffs = st_pms - indep_pm

lcegp_mse = lcegp_diffs.pow(2).mean(dim=0)
st_mse = st_diffs.pow(2).mean(dim=0)

print(f"lcegp avg std: {lcegp_stds.mean()}")
print(f"st avg std: {st_stds.mean()}")

print(f"lcegp avg mse: {lcegp_mse.mean()}")
print(f"st avg mse: {st_mse.mean()}")

lcegp_mse_normalized = lcegp_mse.sqrt() / indep_pm.abs()
st_mse_normalized = st_mse.sqrt() / indep_pm.abs()

print(f"lcegp avg normalized mse: {lcegp_mse_normalized.mean()}")
print(f"st avg normalized mse: {st_mse_normalized.mean()}")

/home/saitcakmak/anaconda3/envs/contextual_rs/lib/python3.8/site-packages/gpytorch/utils/cholesky.py:51: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(f"A not p.d., added jitter of {jitter_new:.1e} to the diagonal", NumericalWarning)
/home/saitcakmak/anaconda3/envs/contextual_rs/lib/python3.8/site-packages/gpytorch/utils/cholesky.py:51: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(f"A not p.d., added jitter of {jitter_new:.1e} to the diagonal", NumericalWarning)
/home/saitcakmak/anaconda3/envs/contextual_rs/lib/python3.8/site-packages/gpytorch/utils/cholesky.py:51: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(f"A not p.d., added jitter of {jitter_new:.1e} to the diagonal", NumericalWarning)


lcegp avg std: 0.30070069432258606
st avg std: 0.0
lcegp avg mse: 3.3249924182891846
st avg mse: 1.4980558156967163
lcegp avg normalized mse: 0.04955904558300972
st avg normalized mse: 0.029212482273578644


So, re-tries reduce the "std", i.e., the variance between the posterior means across
replications, but at least in this specific run, it lead to an increase in MSE. That's
not that great.

Could inferred noise level be playing a role here? Maybe LCEGP attributes too much to
noise.

In [7]:
def get_fitted_noise(replications: int) -> Tuple[Tensor, Tensor]:
    lcegp_noise = torch.zeros(replications)
    st_noise = torch.zeros(replications)
    for rep in range(replications):
        lcegp = LCEGP(
            lcegp_train_X, train_Y, [0], outcome_transform=Standardize(m=1)
        )
        mll = ExactMarginalLogLikelihood(lcegp.likelihood, lcegp)
        custom_fit_gpytorch_model(mll)

        stgp = SingleTaskGP(
            st_train_X, train_Y, outcome_transform=Standardize(m=1)
        )
        mll = ExactMarginalLogLikelihood(stgp.likelihood, stgp)
        fit_gpytorch_model(mll)

        lcegp_noise[rep] = lcegp.likelihood.noise
        st_noise[rep] = stgp.likelihood.noise
    return lcegp_noise, st_noise

In [8]:
lcegp_noise, st_noise = get_fitted_noise(30)
print(f"LCEGP noise, mean: {lcegp_noise.mean()}, std: {lcegp_noise.std()}")
print(f"STGP noise, mean: {st_noise.mean()}, std: {st_noise.std()}")

LCEGP noise, mean: 0.011341067031025887, std: 0.0010141489328816533
STGP noise, mean: 0.011098301038146019, std: 0.0


Noise doesn't seem to be the reason here. Let's look into the arm covariance matrix.

Note that what we calculate below is the prior covariance.

In [9]:
from botorch.optim.fit import fit_gpytorch_torch
from gpytorch.distributions import MultivariateNormal

def get_arm_covariance(
    replications: int, fit_tries: int = 1, adam: bool = False, init_method: str = None,
) -> Tuple[Tensor, Tensor]:
    arm_covars = list()
    for rep in range(replications):
        torch.manual_seed(rep)
        lcegp = LCEGP(
            lcegp_train_X, train_Y, [0], outcome_transform=Standardize(m=1)
        )
        emb_layer = lcegp.emb_layers[0]
        if init_method == "rand":
            new_weight = torch.rand_like(emb_layer.weight)
            emb_layer.weight = torch.nn.Parameter(
                new_weight, requires_grad=True
            )
        elif init_method == "gp":
            covar = lcegp.emb_covar_module
            latent_covar = covar(
                torch.linspace(
                    0.0,
                    1.0,
                    num_arms,
                )
            ).add_jitter(1e-4)
            latent_dist = MultivariateNormal(
                torch.zeros(num_arms),
                latent_covar,
            )
            latent_sample = latent_dist.sample().reshape(emb_layer.weight.shape)
            emb_layer.weight = torch.nn.Parameter(
                latent_sample, requires_grad=True
            )
            # They also register the distribution as a prior but I am not sure how to
            # do that there.
        mll = ExactMarginalLogLikelihood(lcegp.likelihood, lcegp)
        if adam:
            custom_fit_gpytorch_model(
                mll,
                optimizer=fit_gpytorch_torch,
                num_retries=fit_tries,
                options={"disp": False},
            )
        else:
            custom_fit_gpytorch_model(mll, num_retries=fit_tries)

        arms = arm_set.long().view(-1)
        x_emb = lcegp.emb_layers[0](arms)
        covar_emb = lcegp.emb_covar_module(x_emb)
        arm_covars.append(covar_emb.evaluate())
    return torch.stack(arm_covars)

In [10]:
arm_covariances = get_arm_covariance(30)

print(f"mean covariance: {arm_covariances.mean(dim=0)}")
print(f"covariance std: {arm_covariances.std(dim=0)}")

mean covariance: tensor([[1.0000, 0.7754, 0.7982, 0.5545],
        [0.7754, 1.0000, 0.7179, 0.5509],
        [0.7982, 0.7179, 1.0000, 0.6659],
        [0.5545, 0.5509, 0.6659, 1.0000]], grad_fn=<MeanBackward1>)
covariance std: tensor([[0.0000, 0.3944, 0.2867, 0.2996],
        [0.3944, 0.0000, 0.3289, 0.3299],
        [0.2867, 0.3289, 0.0000, 0.3613],
        [0.2996, 0.3299, 0.3613, 0.0000]], grad_fn=<StdBackward1>)


With Adam

In [11]:
arm_covariances = get_arm_covariance(30, 1, True)

print(f"mean covariance: {arm_covariances.mean(dim=0)}")
print(f"covariance std: {arm_covariances.std(dim=0)}")

mean covariance: tensor([[1.0000, 0.7944, 0.8041, 0.5593],
        [0.7944, 1.0000, 0.7910, 0.5557],
        [0.8041, 0.7910, 1.0000, 0.7569],
        [0.5593, 0.5557, 0.7569, 1.0000]], grad_fn=<MeanBackward1>)
covariance std: tensor([[0.0000, 0.3967, 0.2709, 0.2243],
        [0.3967, 0.0000, 0.3376, 0.2775],
        [0.2709, 0.3376, 0.0000, 0.3102],
        [0.2243, 0.2775, 0.3102, 0.0000]], grad_fn=<StdBackward1>)


Indeed, the fitted covariance is quite variable between different tries.

Let's try the same thing with multiple fit tries.

In [12]:
arm_covariances = get_arm_covariance(30, 10)

print(f"mean covariance: {arm_covariances.mean(dim=0)}")
print(f"covariance std: {arm_covariances.std(dim=0)}")


/home/saitcakmak/anaconda3/envs/contextual_rs/lib/python3.8/site-packages/gpytorch/utils/cholesky.py:51: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(f"A not p.d., added jitter of {jitter_new:.1e} to the diagonal", NumericalWarning)


mean covariance: tensor([[1.0000, 0.9974, 0.9588, 0.8130],
        [0.9974, 1.0000, 0.9720, 0.8429],
        [0.9588, 0.9720, 1.0000, 0.9065],
        [0.8130, 0.8429, 0.9065, 1.0000]], grad_fn=<MeanBackward1>)
covariance std: tensor([[0.0000, 0.0028, 0.0431, 0.1790],
        [0.0028, 0.0000, 0.0431, 0.1577],
        [0.0431, 0.0431, 0.0000, 0.1215],
        [0.1790, 0.1577, 0.1215, 0.0000]], grad_fn=<StdBackward1>)


Multiple fit tries does help reduce the variability, but it still does not eliminate it.

With Adam

In [13]:
arm_covariances = get_arm_covariance(30, 10, True)

print(f"mean covariance: {arm_covariances.mean(dim=0)}")
print(f"covariance std: {arm_covariances.std(dim=0)}")

mean covariance: tensor([[1.0000, 0.9625, 0.9384, 0.8415],
        [0.9625, 1.0000, 0.9644, 0.8578],
        [0.9384, 0.9644, 1.0000, 0.9506],
        [0.8415, 0.8578, 0.9506, 1.0000]], grad_fn=<MeanBackward1>)
covariance std: tensor([[0.0000, 0.1747, 0.1362, 0.1227],
        [0.1747, 0.0000, 0.0606, 0.1326],
        [0.1362, 0.0606, 0.0000, 0.0425],
        [0.1227, 0.1326, 0.0425, 0.0000]], grad_fn=<StdBackward1>)


That was a long shot, and the results are as expected. The optimizer is not the problem
. We just have too many local optima, so the results really depend on the initialization.

## One last thing we could try is to initialize the embedding as a prior draw?

So, HOGP uses either `torch.rand` or draws from prior to initialize latents. We let
`Embedding` handle the initialization, and use `torch.randn` when we do multiple fit
tries. Let's give their methods a shot.

In [14]:
arm_covariances = get_arm_covariance(30, 1, False, "rand")

print(f"mean covariance: {arm_covariances.mean(dim=0)}")
print(f"covariance std: {arm_covariances.std(dim=0)}")

mean covariance: tensor([[1.0000, 0.9669, 0.8718, 0.6477],
        [0.9669, 1.0000, 0.8624, 0.6644],
        [0.8718, 0.8624, 1.0000, 0.8314],
        [0.6477, 0.6644, 0.8314, 1.0000]], grad_fn=<MeanBackward1>)
covariance std: tensor([[0.0000, 0.1403, 0.1953, 0.2896],
        [0.1403, 0.0000, 0.2349, 0.2862],
        [0.1953, 0.2349, 0.0000, 0.2118],
        [0.2896, 0.2862, 0.2118, 0.0000]], grad_fn=<StdBackward1>)


In [15]:
arm_covariances = get_arm_covariance(30, 1, True, "rand")

print(f"mean covariance: {arm_covariances.mean(dim=0)}")
print(f"covariance std: {arm_covariances.std(dim=0)}")

mean covariance: tensor([[1.0000, 0.9617, 0.9107, 0.6707],
        [0.9617, 1.0000, 0.9178, 0.7139],
        [0.9107, 0.9178, 1.0000, 0.8982],
        [0.6707, 0.7139, 0.8982, 1.0000]], grad_fn=<MeanBackward1>)
covariance std: tensor([[0.0000, 0.1776, 0.0167, 0.0299],
        [0.1776, 0.0000, 0.1615, 0.1043],
        [0.0167, 0.1615, 0.0000, 0.0130],
        [0.0299, 0.1043, 0.0130, 0.0000]], grad_fn=<StdBackward1>)


This one is interesting. `rand` and `Adam` coupled together yield significantly less
variance.

In [16]:
arm_covariances = get_arm_covariance(30, 1, False, "gp")

print(f"mean covariance: {arm_covariances.mean(dim=0)}")
print(f"covariance std: {arm_covariances.std(dim=0)}")

mean covariance: tensor([[1.0000, 0.9926, 0.9423, 0.7525],
        [0.9926, 1.0000, 0.9497, 0.7720],
        [0.9423, 0.9497, 1.0000, 0.8999],
        [0.7525, 0.7720, 0.8999, 1.0000]], grad_fn=<MeanBackward1>)
covariance std: tensor([[0.0000, 0.0191, 0.0484, 0.2159],
        [0.0191, 0.0000, 0.0786, 0.2100],
        [0.0484, 0.0786, 0.0000, 0.1208],
        [0.2159, 0.2100, 0.1208, 0.0000]], grad_fn=<StdBackward1>)


In [17]:
arm_covariances = get_arm_covariance(30, 1, True, "gp")

print(f"mean covariance: {arm_covariances.mean(dim=0)}")
print(f"covariance std: {arm_covariances.std(dim=0)}")

mean covariance: tensor([[1.0000, 0.9937, 0.9212, 0.6869],
        [0.9937, 1.0000, 0.9554, 0.7477],
        [0.9212, 0.9554, 1.0000, 0.8973],
        [0.6869, 0.7477, 0.8973, 1.0000]], grad_fn=<MeanBackward1>)
covariance std: tensor([[0.0000, 0.0042, 0.0253, 0.0454],
        [0.0042, 0.0000, 0.0177, 0.0347],
        [0.0253, 0.0177, 0.0000, 0.0152],
        [0.0454, 0.0347, 0.0152, 0.0000]], grad_fn=<StdBackward1>)


So, using `Adam` with `rand` or `gp` does reduce the variability. Let's give it a shot
and see how it goes.